In [ ]:
#we used cnn architecture with sigmoid and binary-crossentropy, and 10 epochs and 256 * 256 
from multiprocessing.pool import ThreadPool
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
%matplotlib inline
import os
import random
import glob
import itertools
from keras.utils import to_categorical
import seaborn as sns
import tensorflow as tf
from keras import regularizers

In [ ]:
import pandas as pd
random.seed(10)
trainLabels = pd.read_csv("../input/diabetic-retinopathy-detection/trainLabels.csv.zip")
trainLabels.head()


In [ ]:
from PIL import Image
from keras.preprocessing import image
import os
import numpy as np
import zipfile 
# resize the image to 300x300
img_rows, img_cols = 350,350

listing = os.listdir("../input/diabetic-retinopathy-detection") 
listing.remove("trainLabels.csv.zip")
listing.remove("sampleSubmission.csv.zip")


immatrix = []
imlabel = []

    
for file in listing:
    print(file)
    zf = zipfile.ZipFile("../input/diabetic-retinopathy-detection/" + file)
    print(zf.filename)
    #zf.extractall()
    imageslist = zf.namelist()
    
    
    for imagefile in imageslist: 
        fileName = os.path.splitext(imagefile)[0]
        print(fileName)
        print(trainLabels.loc[image==fileName][level])
        imlabel.append(str(trainLabels.loc[image==fileName][level]))
        im = Image.open(imagefile)
        img = np.array(im.resize((img_rows,img_cols)))
        immatrix.append(np.array(img))

In [ ]:
from sklearn.utils import shuffle

data,label = shuffle(immatrix, imlabel, random_state=42)
train_data = [data,label]

import matplotlib.pyplot as plt

plt.hist(label)
plt.show()

In [ ]:
#split the dataset into 60% training , 20% test, 20% validation dataset 
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data[0], train_data[1], test_size = 0.1, random_state = 42)


print(np.array(x_train).shape)
print(np.array(y_train).shape)

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(np.array(y_train), 5)
y_test = np_utils.to_categorical(np.array(y_test), 5)


x_train = np.array(x_train).astype("float32")/255.
x_test = np.array(x_test).astype("float32")/255.


print(np.array(y_train).shape)

In [ ]:
#specify the green channel of the images..
im = Image.fromarray(immatrix[1],'RGB')
print("level:",imlabel[1])
im

In [ ]:
from sklearn.utils import shuffle

data,label = shuffle(immatrix, imlabel, random_state=42)
train_data = [data,label]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(label)
plt.show()

In [ ]:
from keras.applications import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNet
from keras.applications import InceptionResNetV2

def create_model(input_shape, n_out):
    pretrain_model = InceptionResNetV2(
        include_top=False, 
        weights='imagenet', 
        input_shape=input_shape)    
    input_tensor = Input(shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = pretrain_model(bn)
    x = Conv2D(128, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model
#x = GlobalAveragePooling2D()(backbone.output)
#outputs = Dense(5, activation="softmax")(x)


In [ ]:
keras.backend.clear_session()

model = create_model(
    input_shape=x_train[0].shape, 
    n_out=5)

model.summary()

In [ ]:
model.save_weights('inceptionResNetv2_model_wieghts.h5')
model.save('inceptionResNetv2_model_keras.h5')
model_json=model.to_json()
with open("model.json", "w")as json_file:
    json_file.write(model_json)



In [ ]:
def f1(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def show_history(history):
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('f1')
    ax[1].plot(history.epoch, history.history["f1"], label="Train f1")
    ax[1].plot(history.epoch, history.history["val_f1"], label="Validation f1")
    ax[2].set_title('acc')
    ax[2].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[2].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()




In [ ]:
#from keras.optimizers import SGD
#from tensorflow.keras.optimizers import RMSprop
import torchvision
from numpy import *
history_InceptionResNetV2 =  model.compile(optimizer=Adam(1e-3)

                    , loss='binary_crossentropy'
                    , metrics=['acc',f1])

In [ ]:
def brightness_adjustment(img):
    # turn the image into the HSV space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # creates a random bright
    ratio = .5 + np.random.uniform()
    # convert to int32, so you don't get uint8 overflow
    # multiply the HSV Value channel by the ratio
    # clips the result between 0 and 255
    # convert again to uint8
    hsv[:,:,2] =  np.clip(hsv[:,:,2].astype(np.int32) * ratio, 0, 255).astype(np.uint8)
    # return the image int the BGR color space
    return cv2.cvtColor(hsv, cv2.COLOR_HSV)

In [ ]:
#use augmentation to increase the number of dataset and solve overfitting problems..
from keras.preprocessing.image import ImageDataGenerator
import numpy
X_train = numpy.array(x_train, copy=True) 
Y_train = numpy.array(y_train, copy=True) 
shift = 0.2
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=360,
   preprocessing_function=brightness_adjustment,
                                   width_shift_range=shift,
                                   height_shift_range=shift, shear_range=0.2,
                                   zoom_range=0.2, channel_shift_range=4.,
                                   horizontal_flip=True, vertical_flip=True,
                                   rescale=1. /255,
#zca_whitening = True                                 
fill_mode='nearest') 

datagen.fit(x_train)

#print(type(X_train2))
#print(type(x_train))

# Concatenating the old data with the augmented data
train_x  = numpy.concatenate((x_train, X_train), axis=0)
train_y  = numpy.concatenate((y_train, Y_train), axis=0)
print(train_x.shape)
print(train_y.shape)

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

best_save_model_file = '../working/inception_model.h5'
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=20,
                           verbose=1,
                           min_delta=0.00001,
                           mode='min'),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=2,
                               verbose=1,
                               min_delta=0.0001,
                               mode='min'),
             ModelCheckpoint(monitor='val_loss',save_weights_only=True,
                             filepath=best_save_model_file,
                             save_best_only=True,
                             mode='min') ,
             ]

In [ ]:
history_InceptionResNetV2 =model.fit(train_x, train_y, batch_size = 32, epochs=100,validation_split=0.1, verbose=1,
                               shuffle=True)


In [ ]:
#history_inceptionv3= model.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(lr=0.0001), metrics=['acc', f1])

In [ ]:
#use augmentation to increase the number of dataset and solve overfitting problems..
from keras.preprocessing.image import ImageDataGenerator
import numpy
X_test = numpy.array(x_test, copy=True) 
Y_test = numpy.array(y_test, copy=True) 
shift = 0.2
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=360,
   preprocessing_function=brightness_adjustment,
                                   width_shift_range=shift,
                                   height_shift_range=shift, shear_range=0.2,
                                   zoom_range=0.2, channel_shift_range=4.,
                                   horizontal_flip=True, vertical_flip=True,
                                   rescale=1. /255,
#zca_whitening = True                                 
fill_mode='nearest') 

datagen.fit(x_test)

#print(type(X_train2))
#print(type(x_train))

# Concatenating the old data with the augmented data
test_x  = numpy.concatenate((x_test, X_test), axis=0)
test_y  = numpy.concatenate((y_test, Y_test), axis=0)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred_Y = model.predict(test_x, batch_size = 16, verbose = True)
pred_Y_cat = np.argmax(pred_Y, -1)
test_Y_cat = np.argmax(test_y, -1)
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(test_Y_cat, pred_Y_cat)))
print(classification_report(test_Y_cat, pred_Y_cat))


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_Y_cat, pred_Y_cat), 
            annot=True, fmt="d", cbar = False, cmap = plt.cm.Blues, vmax = test_x.shape[0]//16)

In [ ]:
show_history(history_InceptionResNetV2)

In [ ]:
t=154
test=test_x[t].reshape(-1,350,350,3)
arr=model.predict(test)
arr=arr.flatten()

maxm=arr[0]
for i in range(0,5):
    if(arr[i]>=maxm):
        predicted_label=i+1
        maxm=arr[i]
for i in range(0,5):
    if(int(test_y[t][i])==1):
        actual_label=i+1
        break
name={1:"NO level of Diabetic Retinopathy",2:"MILD level of Diabetic Retinopathy",3:"MODERATE level of Diabetic Retinopathy",4:"SEVERE level of Diabetic Retinopathy",5:"PROLIFERATIVE level of Diabetic Retinopathy"
}

import matplotlib.pyplot as plt
plt.imshow(x_test[t])
plt.show

print("Predicted  : The patient is having " + name[predicted_label])
print("Expected   : The patient should have "+name[actual_label])

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred_Y = model.predict(test_x, batch_size = 16, verbose = True)
pred_Y_cat = np.argmax(pred_Y, -1)
test_Y_cat = np.argmax(test_y, -1)
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(test_Y_cat, pred_Y_cat)))
print(classification_report(test_Y_cat, pred_Y_cat))


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_Y_cat, pred_Y_cat), 
            annot=True, fmt="d", cbar = False, cmap = plt.cm.Blues, vmax = test_x.shape[0]//16)

In [ ]:
score = model.evaluate(test_x,test_y, verbose=0)
print(score)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
sick_vec = test_Y_cat>0
sick_score = np.sum(pred_Y[:,1:],1)
fpr, tpr, _ = roc_curve(sick_vec, sick_score)
fig, ax1 = plt.subplots(1,1, figsize = (6, 6), dpi = 150)
ax1.plot(fpr, tpr, 'b.-', label = 'Model Prediction (AUC: %2.2f)' % roc_auc_score(sick_vec, sick_score))
ax1.plot(fpr, fpr, 'g-', label = 'Random Guessing')
ax1.legend()
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate');


In [ ]:
score = model.evaluate(test_x, test_y , verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
show_history(history_inceptionvResNetV2)

In [ ]:
results = pd.DataFrame({'ImageId': pd.Series(range(1, len(test_Y_cat) + 1)), 'Label': pd.Series(pred_Y_cat)})
results.to_csv('results.csv', index=False)
results.head(200)

from sklearn import metrics
print(metrics.accuracy_score(test_Y_cat, pred_Y_cat))
confusion = metrics.confusion_matrix(test_Y_cat, pred_Y_cat)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]


#Also known as "Misclassification Rate"

print((FP + FN) / float(TP + TN + FP + FN))
print(1 - metrics.accuracy_score(test_Y_cat, pred_Y_cat))

#Sensitivity: When the actual value is positive, how often is the prediction correct?

###How "sensitive" is the classifier to detecting positive instances?
#Also known as "True Positive Rate" or "Recall"

print(TP / float(TP + FN))
print(metrics.recall_score(test_Y_cat, pred_Y_cat))

#Specificity: When the actual value is negative, how often is the prediction correct?

#How "specific" (or "selective") is the classifier in predicting positive instances?

print(TN / float(TN + FP))



#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?


print(FP / float(TN + FP))